%pip install -r requirments.txt --force

https://colab.research.google.com/drive/1T3JjUyyaEUZN8hUVymyyvleDjR8ZJehE#scrollTo=lJY6Pz51PCM3


In [2]:

from flask_socketio import SocketIO, emit, join_room, leave_room
from flask import *
from flask_login import *
from flask_session import Session
from pypdf import PdfReader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from langchain_community.vectorstores import FAISS
# from datetime import timedelta
from langchain.chat_models import ChatOllama
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain

In [3]:
model = ChatOllama(model='meta-llama-3.1')
#Meta-Llama-3.1-8B-Instruct-Q4_K_M.gguf
#https://huggingface.co/lmstudio-community/Meta-Llama-3.1-8B-Instruct-GGUF/tree/main

app = Flask(__name__)
app.secret_key = "mkqw2o0@#mk12!mk3"

app.config['SESSION_TYPE'] = 'filesystem' 
# app.config['SESSION_PERMANENT'] = False
# app.config['SESSION_USE_SIGNER'] = True
# app.config['SESSION_KEY_PREFIX'] = 'myapp_session:'
Session(app)
socketio = SocketIO(app)
lm = LoginManager()
lm.init_app(app)

ngrok = ''

from langchain_community.embeddings import HuggingFaceEmbeddings

embedding_model = HuggingFaceEmbeddings(
    model_name="BAAI/bge-m3",
    model_kwargs = {'device': 'cpu'}, # 모델이 CPU에서 실행되도록 설정. GPU를 사용할 수 있는 환경이라면 'cuda'로 설정할 수도 있음
    encode_kwargs = {'normalize_embeddings': True}, # 임베딩 정규화. 모든 벡터가 같은 범위의 값을 갖도록 함. 유사도 계산 시 일관성을 높여줌
)

@lm.user_loader
def user_loader(userId):
    userInfo = User.get_user_info(userId)
    return User(userInfo)

@lm.unauthorized_handler
def unauthorized():
    return redirect('/')

#region CLASS

class User(UserMixin):
    def __init__(self, info):
        self.info = info
    
    #region getter

    def get_id(self):
        return self.info['userId']

    #endregion
    
    @staticmethod
    def get_user_info(userId):
        return {'userId': userId}

#endregion CLASS

#region route

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/chatroom', methods = ['post'])
def chatroom():
    params = request.form
    userId = params['userId']
    roomId = params['roomId']
    
    session['userId'] = userId
    session['roomId'] = roomId
    
    return render_template(
        'chat.html',
        userId = userId,
        roomId = roomId
    )

@app.route('/chatbot')
def chatbot():
    if 'processed_pdf' in session:
        return 'test msg'
    return render_template('RAG.html')

@app.route('/login', methods = ['post'])
def login():
    params = request.get_json()
    userId = params['userId']
    
    userInfo = User.get_user_info(userId)
    login_user(User(userInfo))
    return jsonify({'result': 1})

@app.route('/logout')
def logout():
    logout_user()
    return redirect('/')

@app.route('/pdfprocess', methods=['post'])
def pdfprocess():
    files = [request.files[i] for i in request.files]
    
    text_sum = ''
    
    for file in files:
        reader = PdfReader(file)
        for page in reader.pages: #페이지 별로 텍스트 추출
            text = page.extract_text()
            corrected_text = text.encode('utf-8', errors='ignore').decode('utf-8') #인코딩 오류 무시 및 텍스트 누적
            text_sum += corrected_text +'\n'

    # print(text_sum)
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    splits = text_splitter.split_text(text_sum)
    # print(splits)
    vectorstore = FAISS.from_texts(
        splits,
        embedding = embedding_model
    )
    
    vectorstore.save_local('test')
    print('processed')
    #region
    
    # reader = PdfReader(f)
    # # 총 페이지 수 확인
    # num_pages = len(reader.pages)

    # # 모든 페이지 텍스트 추출
    # all_text = ""
    # for page_num in range(num_pages):
    #     page = reader.pages[page_num]
    #     text = page.extract_text()

    #     # 인코딩 오류 무시 및 텍스트 누적
    #     corrected_text = text.encode('utf-8', errors='ignore').decode('utf-8')
    #     all_text += corrected_text + "\n"  # 페이지 간 텍스트 구분을 위해 줄 바꿈 추가

    # # 추출한 텍스트를 txt 파일로 저장
    # output_file_path = './save.txt'
    # with open(output_file_path, 'w', encoding='utf-8') as output_file:
    #     output_file.write(all_text)
    
    #endregion
    return jsonify({'result': 1})

@app.route('/question', methods=['POST'])
def question():
    vectorstore = FAISS.load_local(
        'test',
        embedding_model,
        allow_dangerous_deserialization=True,
    )
    params = request.get_json()
    question = params['question']
    
    memory = ConversationBufferMemory(
        memory_key='chat_history',
        return_messages=True,
    )

    conversation_chain = ConversationalRetrievalChain.from_llm(
        llm=model,
        retriever=vectorstore.as_retriever(),
        memory=memory,
    )
    res = conversation_chain({'question': question})
    print(res)
    
    return jsonify({'result': res['chat_history'][1].content})

#endregion route

#region socket

@socketio.on('joined', namespace = '/chatroom')
def chat_joined(d):
    roomId = session.get('roomId')
    print(d, 'joined', roomId)
    join_room(roomId)

@socketio.on('msg', namespace = '/chatroom')
def socket_msg(d):
    roomId = session.get('roomId')
    print('msg', d)
    socketio.emit('msg', d, namespace = '/chatroom', room = roomId)

#endregion socket

if __name__ == '__main__':
    print('http://192.168.219.51:5100')
    socketio.run(app = app, host = '192.168.219.51', port = 5100, allow_unsafe_werkzeug = True)

c:\Users\SMHRD\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
c:\Users\SMHRD\anaconda3\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


http://192.168.219.51:5100


Ignoring wrong pointing object 9 0 (offset 0)
Ignoring wrong pointing object 11 0 (offset 0)
Ignoring wrong pointing object 21 0 (offset 0)
Ignoring wrong pointing object 25 0 (offset 0)


processed


c:\Users\SMHRD\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


{'question': '자연어 처리 강사님을 확인해줘', 'chat_history': [HumanMessage(content='자연어 처리 강사님을 확인해줘'), AIMessage(content='민재홍  / Min JaehongR을 활용한  공공데이터  분석 강사입니다')], 'answer': '민재홍  / Min JaehongR을 활용한  공공데이터  분석 강사입니다'}
processed
{'question': '문서 내용을 바탕으로 보고서를 작성해야하는데 제목을 만들어줘', 'chat_history': [HumanMessage(content='문서 내용을 바탕으로 보고서를 작성해야하는데 제목을 만들어줘'), AIMessage(content='리버스 엔지니어링 주제 선정에 대한 안내입니다')], 'answer': '리버스 엔지니어링 주제 선정에 대한 안내입니다'}
{'question': '보고서 목차를 만들어줘', 'chat_history': [HumanMessage(content='보고서 목차를 만들어줘'), AIMessage(content='1. 리버스 엔지니어링\r\n2. 전자 상거래\r\n3. 대시보드 (데이터 시각화)\r\n4. 챌린지 포인트 (OpenAI 사용')], 'answer': '1. 리버스 엔지니어링\r\n2. 전자 상거래\r\n3. 대시보드 (데이터 시각화)\r\n4. 챌린지 포인트 (OpenAI 사용'}
